In [ ]:
// #r ".\binaries\bossspad.dll"
// #r ".\binaries\XNSEC.dll"

#r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
//#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
Init();

## Open Database

In [ ]:
static var myDb = OpenOrCreateDatabase(@"C:\Databases\BoSSS_DB");
// static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\Database_HeatedCavity1e5_finer___223");

In [ ]:
myDb.Sessions[0].KeysAndQueries["Grid:NoOfCells"]

256

In [ ]:
BoSSSshell.WorkflowMgm.Init("HeatedSquareCavity");

Project name is set to 'HeatedSquareCavity'.
NotSupportedException caught during creation/opening of default database: `AllowedDatabasesPaths` not specified, unable to create Database (should be specified in ~/.BoSSS/etc/BatchProcessorConfig.json)..


In [ ]:
myDb.Sessions[0].Export().Do()

Starting export process... Data will be written to the directory: C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\BackwardFacingStep__empty-session-name__27386e7c-c061-4eb5-902a-cb02276a78d6


C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\BackwardFacingStep__empty-session-name__27386e7c-c061-4eb5-902a-cb02276a78d6

In [ ]:
myDb.Sessions[0].Timesteps

#0:  { Time-step: 0; Physical time: 0s; Fields: Phi, PhiDG, VelocityX, VelocityY, Pressure, Temperature, MassFraction0, Residual-MomentumX, Residual-MomentumY, Residual-ContiEq, Residual-HeatEq, Residual-SpeciesMassBalance0, ThermodynamicPressure, Density, MixtureHeatCapacity, VelocityX@Phi, VelocityY@Phi, Velocity0X_Mean, Velocity0Y_Mean; Name:  }
#1:  { Time-step: 1; Physical time: 1.7976931348623158E+304s; Fields: Phi, PhiDG, VelocityX, VelocityY, Pressure, Temperature, MassFraction0, Residual-MomentumX, Residual-MomentumY, Residual-ContiEq, Residual-HeatEq, Residual-SpeciesMassBalance0, ThermodynamicPressure, Density, MixtureHeatCapacity, VelocityX@Phi, VelocityY@Phi, Velocity0X_Mean, Velocity0Y_Mean; Name:  }


In [ ]:
// var rightvalue = sess.KeysAndQueries["NusseltNumber1"];
// var residualValue = sess.KeysAndQueries["NusseltNumber2"];

## Create grid

In [ ]:
// int[] Resolutions = new int[]{ 2,3,4,5,6,7 }; //new int[]{ 10, 20 };
// int[] DGdegree = new int[]{ 1,2,3,4 };
int[] Resolutions = new int[]{5 }; //new int[]{ 10, 20 };
int[] DGdegree = new int[]{ 5 };
int[] nCells  =  Resolutions.Select(r => (int)(Math.Pow(2,r+1))).ToArray();

In [ ]:
public static class GridFactory {
    
 
    public static double[] GetXNodes(int Res) {
 
        var xNodes = GenericBlas.SinLinSpacing(-0.5, 0.5, 0, Res + 1);
        return xNodes;
    }
 
    static double[] GetYNodes(int Res) {
        double[] yNodes =  GenericBlas.SinLinSpacing(-0.5, 0.5,0, Res + 1);
        return yNodes;
    }
 
    public static Grid2D GenerateGrid(int Res) { 
        var xNodes = GetXNodes(Res);
        var yNodes = GetYNodes(Res);
        var grd    = Grid2D.Cartesian2DGrid(xNodes, yNodes);
        grd.EdgeTagNames.Add(1, "NoSlipNeumann");
        grd.EdgeTagNames.Add(2, "wall_tempfixed_left");
        grd.EdgeTagNames.Add(3, "wall_tempfixed_right");
        grd.DefineEdgeTags( delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            //Edge tags
            //1: Adiabatic no slip wall
            //2: Temperature fixed no slip wall

            //right cold wall 
            if (Math.Abs(x - 0.5) < 1e-8)
                return 3;

            //bottom adiabatic Wall
            if (Math.Abs(y - 0.5 ) < 1e-8)
                return 1;

            // left hot wall
            if (Math.Abs(x + 0.5) < 1e-8)
                return 2;

            //top adiabatic Wall
            if (Math.Abs(y + 0.5 ) < 1e-8)
                return 1;

            else throw new ArgumentOutOfRangeException();
        });
        
        bool force = true; 
        myDb.SaveGrid(ref grd, force );
        
        return grd;
     }
 
 }

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double Th, double Tc, double Froude) {
        using(var stw = new System.IO.StringWriter()) {
           
           stw.WriteLine("static class BoundaryValues {");
           stw.WriteLine("  static public double VelX(double[] X) {");
           stw.WriteLine("    return 0.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double VelY(double[] X) {");
           stw.WriteLine("    return 0.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double TemperatureHot(double[] X) {");
           stw.WriteLine("    return 1.6;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double TemperatureCold(double[] X) {");
           stw.WriteLine("    return 0.4;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double One(double[] X) {");
           stw.WriteLine("    return  1.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double Zero(double[] X) {");
           stw.WriteLine("    return  0.0;");
           stw.WriteLine("  }");

           stw.WriteLine(" static public double InitialPressure(double[] X) { ");
            stw.WriteLine("         return (-1)* X[1] / ("+Froude * Froude +") ;");
            stw.WriteLine("    }");
            stw.WriteLine("}"); 
           return stw.ToString();
        }
    }
   
    static public Formula Get_VelX(double Th, double Tc , double Froude) {
        return new Formula("BoundaryValues.VelX", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }

    static public Formula Get_VelY(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.VelY", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }
    static public Formula Get_TemperatureHot(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.TemperatureHot", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }

     static public Formula Get_TemperatureCold(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.TemperatureCold", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }

    static public Formula Get_One(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.One", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }
    static public Formula Get_Zero(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.Zero", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }
    static public Formula Get_InitialPressure(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.InitialPressure", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }
}

## Send and run jobs

In [ ]:
var controls = new List<BoSSS.Application.XNSEC.XNSEC_Control>();

In [ ]:
myDb

In [ ]:
var controls = new List<BoSSS.Application.XNSEC.XNSEC_Control>();
double Th = 1.6; double Tc = 0.4;
double Ra = 1e4;
foreach(int dg in DGdegree){
foreach(int Res in Resolutions) {

    var C = new BoSSS.Application.XNSEC.XNSEC_Control();
    C.SetDGdegree(dg);
    var nCells = (int) Math.Pow(2,Res+1);
    C.SetGrid(GridFactory.GenerateGrid(nCells));
    C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Res", Res));
    C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Dgdegree", dg));


    C.EnableMassFractions = false;
    C.NumberOfChemicalSpecies = 1;
    C.ChemicalReactionActive = false;
    C.MatParamsMode = MaterialParamsMode.Sutherland;
    C.physicsMode = PhysicsMode.Combustion;
    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;


    C.LinearSolver = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();
    C.LinearSolver.NoOfMultigridLevels = 5;
    
    // C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;

    C.LinearSolver = LinearSolverCode.direct_pardiso.GetConfig();
    C.NonLinearSolver.ConvergenceCriterion = 1e-11;
    C.LinearSolver.ConvergenceCriterion = 1e-12;
    C.NonLinearSolver.verbose = true;
    C.LinearSolver.verbose = false;
    C.NonLinearSolver.MaxSolverIterations = 50;

    C.PenaltyViscMomentum = 1.0 * 1;
    C.PenaltyHeatConduction = 1.0 * 1;
    C.PhysicalParameters.IncludeConvection = true;

    C.UseSelfMadeTemporalOperator = false;
    C.timeDerivativeEnergyp0_OK = false;
    C.timeDerivativeConti_OK = false;

 
    C.EdgeTagsNusselt = new string[] { "wall_tempfixed_left", "wall_tempfixed_right", "NoSlipNeumann" };

    C.Rayleigh = Ra;
    C.Reynolds = Math.Sqrt(Ra);
    C.Prandtl = 0.71;
    double Fr =Math.Sqrt(2 * C.Prandtl * (1.6 - 0.4) / (1.6 + 0.4)); 
    C.Froude = Fr;
    C.HeatCapacityRatio = 1.4;
    C.T_ref_Sutherland = 600*1+273*0;
    C.ThermodynamicPressureMode = ThermodynamicPressureMode.MassDetermined; // Because its a closed system, i.e. p0 = p0(time)
    C.PhysicalParameters.IncludeConvection = true;
    C.Timestepper_LevelSetHandling         = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
    
    C.SessionName = "NaturalConvection_k" + Res + "_DG" + dg; 
  
    


    C.AddBoundaryValue("NoSlipNeumann", VariableNames.VelocityX,BoundaryValueFactory.Get_VelX(Th, Tc,Fr));
    C.AddBoundaryValue("NoSlipNeumann", VariableNames.VelocityY,BoundaryValueFactory.Get_VelY(Th, Tc,Fr));

    C.AddBoundaryValue("wall_tempfixed_left", VariableNames.Temperature,BoundaryValueFactory.Get_TemperatureHot(Th, Tc,Fr));
    C.AddBoundaryValue("wall_tempfixed_right", VariableNames.Temperature, BoundaryValueFactory.Get_TemperatureCold(Th, Tc,Fr));

    C.AddBoundaryValue("wall_tempfixed_left", VariableNames.MassFraction0, BoundaryValueFactory.Get_One(Th, Tc,Fr));
    C.AddBoundaryValue("wall_tempfixed_right", VariableNames.MassFraction0, BoundaryValueFactory.Get_One(Th, Tc,Fr));





    C.AddInitialValue(VariableNames.VelocityX,  BoundaryValueFactory.Get_Zero(Th, Tc,Fr));
    C.AddInitialValue(VariableNames.VelocityY,BoundaryValueFactory.Get_Zero(Th, Tc,Fr));
    C.AddInitialValue(VariableNames.Pressure,BoundaryValueFactory.Get_InitialPressure(Th, Tc,Fr));

    C.AddInitialValue(VariableNames.Temperature,BoundaryValueFactory.Get_One(Th, Tc,Fr));
    C.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_One(Th, Tc,Fr));

    //C.AddInitialValu(VariableNames.Temperature, X => (Tc - Th) / 1 * X[0] + Th);

    //C.AddInitialValue.Add(VariableNames.Temperature, X => X[0] * X[0] + X[1] * X[1] + 1);
    C.AddInitialValue(VariableNames.ThermodynamicPressure,BoundaryValueFactory.Get_One(Th, Tc,Fr));


    controls.Add(C);
}
}

In [ ]:
Console.WriteLine(controls.Count);

## Run Simulations

In [ ]:
BoSSSshell.ExecutionQueues.ForEach(q => Console.WriteLine(q))

In [ ]:
var myBatch = BoSSSshell.ExecutionQueues[3];

In [ ]:
myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(myDb.Path,""));

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);
foreach(var c in controls) {
    string jobName       = c.SessionName;
    var oneJob           = new Job(jobName, solver);
    int noOfProcs = Convert.ToInt32(c.Paramstudy_CaseIdentification[0].Item2) > 2 ? 4:1;
     noOfProcs = Convert.ToInt32(c.Paramstudy_CaseIdentification[0].Item2) > 4 ? 8:noOfProcs;
    oneJob.NumberOfMPIProcs = noOfProcs;
    oneJob.NumberOfThreads = 1;
    oneJob.SetControlObject(c);
    oneJob.Activate(myBatch); 
}

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

## Post Processing: h-convergence study

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;

In [ ]:
BoSSSshell.WorkflowMgm.hConvergence.Update();

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable; // The session table has to be created again in order to see the results from  hconvergence

In [ ]:
Tab.GetColumnNames().Skip(350)

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns("DGdegree:Temperature","Grid:hMin","Grid:NoOfCells" ,"L2Error_Temperature");
//Extract last point, the one against we are comparing our solutions with
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) !=65536 );
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) !=64 );

In [ ]:
var ErrorPlot = Tab2.ToPlot("Grid:hMin","L2Error_Temperature", "DGdegree:Temperature");
ErrorPlot.LogX = true;
ErrorPlot.LogY = true;

In [ ]:
ErrorPlot.PlotNow()     // No semicolon!!

In [ ]:
ErrorPlot.Regression()

In [ ]:
// string dataPathResultsBase = @"C:\Users\Gutierrez\Documents\data\convergenceProblemData\" ;
string dataPathResultsBase = @"C:\Users\gutierrez\Dropbox\CombustionDGPaper\Paper\data\HSC_convstudy_XNSEC";
string folderpath          = dataPathResultsBase ;// + ending +"LowTemp"+ @"\";
System.IO.Directory.CreateDirectory(folderpath); // Create folder for storing data

In [ ]:
// string[] varnames = new string[]{"VelocityX","VelocityY","Temperature", "Pressure", "ThermodynamicPressure" };// 
string[] varnames = new string[]{"Pressure", "ThermodynamicPressure" };// 

foreach (var varname in varnames) {
    foreach (int pDeg in DGdegree) {
       var pDegSessions = myDb.Sessions.Where(
            Si => (Convert.ToInt32(Si.KeysAndQueries["DGdegree:Velocity*"]) == pDeg) 
         ).ToArray();
        Plot2Ddata pDegPlot =
         pDegSessions.ToEstimatedGridConvergenceData(varname,
           xAxis_Is_hOrDof: true, // false selects DOFs for x-axis 
           normType: NormType.L2_embedded );
            //Saving to a txt file
        pDegPlot.SaveTextFileToPublish(folderpath+"\\"+  varname + "DG" + pDeg, false);
        Console.WriteLine(folderpath+"\\"+  varname + "DG" + pDeg);
    }
}


In [ ]:
myDb.Sessions

In [ ]:
var db2 = OpenOrCreateDatabase(@"C:\Databases\BoSSS_DB");

In [ ]:
db2.Sessions[0]